In [1]:
import json
import pandas

In [ ]:
def get_result_for_label(label, start_idx=0, end_idx=10):
    full_info_list = []
    for run_idx in range(start_idx, end_idx):
        with open(f'{label}_{run_idx}.json') as f:
            autosd_results = json.load(f)
        for arhe_idx, autosd_apr_res in enumerate(autosd_results):
            bug_name = autosd_apr_res['task_id']
            autosd_repair_succ = autosd_apr_res['passed']
            if 'trace' not in autosd_apr_res:
                continue
            synth_process = autosd_apr_res['trace'].split('## Analysis')[-1]
            debugging_done = '<DEBUGGING DONE>' in synth_process
            succ_hypothesis = 'The hypothesis is supported' in synth_process or 'The hypothesis was supported' in synth_process
            full_info_list.append({
                'run_idx': run_idx,
                'humaneval_name': bug_name,
                'arhe_idx': arhe_idx,
                'autosd_repair_succ': autosd_repair_succ,
                'debugging_done': debugging_done,
                'succ_hypothesis': succ_hypothesis,
            })
    return pandas.DataFrame(full_info_list)

In [5]:
# headline results
label = './results/chatgpt002_ZSDloopV4_s3_out'
full_info_df = get_result_for_label(label)
print('AutoSD Fixed:', full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique())

print('Acc of Done:', (full_info_df[full_info_df.debugging_done]['autosd_repair_succ'].sum()/
                      full_info_df.debugging_done.sum()))
print('Acc of Not Done:', (full_info_df[~full_info_df.debugging_done]['autosd_repair_succ'].sum()/
                           (~full_info_df.debugging_done).sum()))

AutoSD Fixed: 189
Acc of Done: 0.8188277087033747
Acc of Not Done: 0.6941508104298802


In [1]:
import pandas as pd
df = pd.read_json("/data/wyk/autosd/arhe/results/chatgpt002_ZSDloopV4_s3_out_0.json")
df.head()

,mutant,mut_op,task_id,failed_tests,samples,trace,prompt_at_repair,passed,fail_tests,ARHE_id
0,"def intersperse(numbers: List[int], delimeter:...",_int_const_changer,HumanEval/5,[{'failing_assertion': ' assert candidate([...,"[\ndef intersperse(numbers: List[int], delimet...",# Debugging Log\n\nAuthored by Linus Torvalds....,# Debugging Log\n\nAuthored by Linus Torvalds....,True,[],0
1,def longest(strings: List[str]) -> Optional[st...,_if_remover,HumanEval/12,[{'failing_assertion': ' assert candidate([...,[\ndef longest(strings: List[str]) -> Optional...,# Debugging Log\n\nAuthored by Linus Torvalds....,# Debugging Log\n\nAuthored by Linus Torvalds....,True,[],1
2,"def sort_numbers(numbers: str) -> str:\n """"...",_int_const_changer,HumanEval/19,[{'failing_assertion': ' assert candidate('...,[\ndef sort_numbers(numbers: str) -> str:\n ...,# Debugging Log\n\nAuthored by Linus Torvalds....,# Debugging Log\n\nAuthored by Linus Torvalds....,True,[],2
3,"def sort_numbers(numbers: str) -> str:\n """"...",_int_const_changer,HumanEval/19,[{'failing_assertion': ' assert candidate('...,[\ndef sort_numbers(numbers: str) -> str:\n ...,# Debugging Log\n\nAuthored by Linus Torvalds....,# Debugging Log\n\nAuthored by Linus Torvalds....,True,[],3
4,"def sort_numbers(numbers: str) -> str:\n """"...",_str_const_changer,HumanEval/19,[{'failing_assertion': ' assert candidate('...,[\ndef sort_numbers(numbers: str) -> str:\n v...,# Debugging Log\n\nAuthored by Linus Torvalds....,# Debugging Log\n\nAuthored by Linus Torvalds....,True,[],4


In [49]:
len(df.loc[4, 'fail_tests'])

5

In [47]:
print(df.loc[4, 'fail_tests'][1])

{'failing_assertion': "    assert candidate('three') == 'three'", 'failing_exception': "<class 'SyntaxError'>"}


In [4]:
print(df.loc[16, 'trace'])

# Debugging Log

Authored by Linus Torvalds.

I use the _scientific method_ to debug the problem below (as written by Zeller, 2009) by describing the hypothesis/prediction/experiment/observation/conclusion. This can be done by:

Hypothesis: An explanation for the buggy behavior behavior. Hypotheses are the key aspect of the approach, and should be detailed and written with care. Hypotheses should build upon all previous information; repeating previous hypotheses is thus strongly discouraged. Some examples are provided below.

 - Example hypothesis 1: "Given that [information], the method is [overall erroneous behavior]. Specifically, I think it is because `c>b` on line 4321 of method `foo` is intended to [desired behavior], but is [erroneous behavior]."
 - Example hypothesis 2: "The previous hypothesis was rejected. we now know `c>b` on line 4321 of the method `foo` is likely not the culprit. Looking elsewhere, perhaps `x.append(y)` on line 4323 of `foo` should do [desired behavior], b

In [2]:
print(df.loc[1, 'prompt_at_repair'])

# Debugging Log

Authored by Linus Torvalds.

I use the _scientific method_ to debug the problem below (as written by Zeller, 2009) by describing the hypothesis/prediction/experiment/observation/conclusion. This can be done by:

Hypothesis: An explanation for the buggy behavior behavior. Hypotheses are the key aspect of the approach, and should be detailed and written with care. Hypotheses should build upon all previous information; repeating previous hypotheses is thus strongly discouraged. Some examples are provided below.

 - Example hypothesis 1: "Given that [information], the method is [overall erroneous behavior]. Specifically, I think it is because `c>b` on line 4321 of method `foo` is intended to [desired behavior], but is [erroneous behavior]."
 - Example hypothesis 2: "The previous hypothesis was rejected. we now know `c>b` on line 4321 of the method `foo` is likely not the culprit. Looking elsewhere, perhaps `x.append(y)` on line 4323 of `foo` should do [desired behavior], b

In [6]:
# LLM-Base results
label = './results/chatgpt002_ZSnoSDloopV4_s0_out'
full_info_df = get_result_for_label(label)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

179

In [7]:
# Codex-AutoSD
label = './results/codex_ZSD_loopV4_s3_results'
full_info_df = get_result_for_label(label)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

120

In [8]:
# Codex LLM-Base
label = './results/codex_ZSnoSD_loopV4_s0_T0'
full_info_df = get_result_for_label(label)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

180

In [7]:
# CodeGen-Big (6B) AutoSD
label = './data/BigCG_ZSDloopV4_s3_results'
full_info_df = get_result_for_label(label)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

0

In [8]:
# CodeGen-Big (6B) LLM-Base
label = './data/autosd/bigcg_results/BigCG_ZSnoSD_s0_results'
full_info_df = get_result_for_label(label, start_idx=3, end_idx=4)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

0

In [9]:
# Fewshot CodeGen-Big (6B) LLM-Base
label = './data/BigCG_FSnoSDnoDef_s0_results'
full_info_df = get_result_for_label(label)
full_info_df[full_info_df.autosd_repair_succ]['arhe_idx'].nunique()

44

In [9]:
# reverse mutator baseline (implementation in reverse_mutator.py)
import numpy as np
base_perf = [83, 75, 85, 90, 82, 84, 82, 88, 79, 84, 86, 86, 93, 81, 85, 87, 81, 85, 92, 84, 95, 91, 87, 81, 93, 89, 84, 80, 89, 84, 86, 84, 85, 87, 83, 87, 90, 85, 82, 75, 84, 85, 88, 89, 91, 84, 93, 82, 82, 91, 83, 88, 77, 89, 82, 85, 88, 92, 91, 92, 84, 89, 87, 89, 93, 88, 89, 88, 85, 85, 78, 86, 85, 90, 86, 82, 84, 80, 87, 89, 91, 82, 89, 77, 88, 89, 88, 82, 91, 84, 86, 88, 81, 80, 79, 81, 89, 85, 88, 90]
print(np.mean(base_perf))
print(np.std(base_perf))

85.77
4.199654747714388
